In [1]:
#目标score(log loss) <0.06127(RANK 131)
#1 数据预处理
#2 模型预处理


In [1]:
#1.1 导入必要的库
#from sklearn.datasets import load_files       
#from keras.utils import np_utils
#from glob import glob

import cv2  #working with images mainly resizing
import numpy as np  #dealing with arrays
import os #dealing with directories  ？？？
from random import shuffle #？？mixing up or currently oredered data that might lead our networ astray in training
from tqdm import tqdm  # a nice pretty percentage bar for tasks，就是个进度条...
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  
%matplotlib inline


ModuleNotFoundError: No module named 'cv2'

In [3]:
#1.2 全局变量
IMG_SIZE = 224
CHANNELS = 3   #train&test inages resized(224*224*3)

LR = 0.01   #learning rate

TRAIN_DIR = 'F:/Temp/train/'
TEST_DIR = 'F:/Temp/test/'

In [4]:
#1.3 导入数据及整理

def label_img(img):
    word_label = img.split('.')[-3]  #以"."为分割，从后(-1)往前数第三个（cat.0.jpg）
    if word_label == 'cat':return [1,0] 
    elif word_label == 'dog':return [0,1]
    
def create_train_and_validation_data ():
    original_data = []
    x_train = []
    y_train = []
    x_validation = []
    y_validation = []
    for img in tqdm(os.listdir(TRAIN_DIR)):  #os.listdir(path)，返回path指定的文件夹包含的文件名字的列表；用tqdm封装显示处理数据的进度
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)   #os.path.join()将多个路径组合后返回
        img = cv2.imread(path,cv2.IMREAD_UNCHANGED)  #cv2.imread()读入图像，cv2.IMREAD_UNCHANGED以读取彩色图片，且包括alpha值(透明度)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        original_data.append([np.array(img),np.array(label)])
    shuffle(original_data)
    x_train, x_validation, y_train, y_validation = train_test_split(np.array(img),np.array(label),test_size=0.25,random_state=42)
    np.save('raw_train_data.npy',original_data)
    np.savea('x_train.npy',x_train)
    np.save('y_train.npy',y_train)
    np.save('x_validation.npy',x_validation)
    np.save('y_validation.npy',y_validation)
    return x_train, y_train, x_validation, y_validation


In [5]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
        shuffle(testing_data)
        np.save('test_data.npy',tesing_data)
        return tesing_data

In [ ]:
x_train, y_train, x_validation, y_validation = create_train_and_validation_data()
# If you have already crenated the dataset:
#train_data = np.load('train_data.npy',allow_pickle=True)

100%|██████████| 25000/25000 [08:43<00:00, 47.78it/s]  


In [ ]:
#2 预处理模型
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import optimizer


In [ ]:
#inception V3的效果好于VGG16,而且基于imageNet训练的模型，比较适合图像识别，xception不基于
inception_weights_path = '../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels.h5'  ###改一下路径，好像可以不用no-top的模型？
InceptionV3_model = Sequential
InceptionV3_model.add(InceptionV3(include_top = False, pooling = 'avg', weights = inception_weights_path)) # this layer will be set no trainablet
InceptionV3_model.add(Dense(2, activation='softmax'))
InceptionV3_model.layers[0].trainable #say not to train  first layer model as it is already trained
InceptionV3_model.summary()

In [ ]:
#complile model
sgd = optimizers.SGD(lr=LR,decay=1e-6, momentum=0.9, nesterov=True) 
#lrm学习率；momentum动量参数，decay每次更新后的学习率衰减值，nesterov确定是否使用nesterov动量
InceptionV3_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#这里可能会报错，使用交叉熵需要把数据换成（data,label）?

In [ ]:
from keras.callbacks import ModelCheckpoint

#train the model
checkpointer = ModelCheckpoint(filepath='',verbose=1,save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=128,validation_data = (x_validation,y_validation),
                 epochs=10,callbacks=[checkpointer],verbose=1,shuffle=True)

#优化空间：之一图像增强；之二对CNN网络最后一层（去除了全连接层）进行fine tuning;之三为多增加全连接层

In [ ]:
plt.figure(1, figsize = (15,8)) 
    
plt.subplot(221)  
plt.plot(hist.history['acc'])  
plt.plot(hist.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
    
plt.subplot(222)  
plt.plot(hist.history['loss'])  
plt.plot(hist.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 

plt.show()